In [4]:
from messis.messis import Messis
from safetensors import safe_open
from transformers import PretrainedConfig
import torch

## Load Messis from Huggingface 🤗 for Inference

In [9]:
config = PretrainedConfig.from_pretrained('crop-classification/messis')
model = Messis.from_pretrained('crop-classification/messis', cache_dir='./hf_cache/')

TemporalViTEncoder | Warning: Unexpected keys in the state dict: ['mask_token', 'decoder_pos_embed', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder_blocks.0.norm1.weight', 'decoder_blocks.0.norm1.bias', 'decoder_blocks.0.attn.qkv.weight', 'decoder_blocks.0.attn.qkv.bias', 'decoder_blocks.0.attn.proj.weight', 'decoder_blocks.0.attn.proj.bias', 'decoder_blocks.0.norm2.weight', 'decoder_blocks.0.norm2.bias', 'decoder_blocks.0.mlp.fc1.weight', 'decoder_blocks.0.mlp.fc1.bias', 'decoder_blocks.0.mlp.fc2.weight', 'decoder_blocks.0.mlp.fc2.bias', 'decoder_blocks.1.norm1.weight', 'decoder_blocks.1.norm1.bias', 'decoder_blocks.1.attn.qkv.weight', 'decoder_blocks.1.attn.qkv.bias', 'decoder_blocks.1.attn.proj.weight', 'decoder_blocks.1.attn.proj.bias', 'decoder_blocks.1.norm2.weight', 'decoder_blocks.1.norm2.bias', 'decoder_blocks.1.mlp.fc1.weight', 'decoder_blocks.1.mlp.fc1.bias', 'decoder_blocks.1.mlp.fc2.weight', 'decoder_blocks.1.mlp.fc2.bias', 'decoder_blocks.2.norm1.weight', 'decoder

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# TODO: Implement inference

## Validate weights loading from .safetensors checkpoint

### Expectation: should initalize the TemporalViTEncoder from the .safetensors checkpoint

In [18]:
# load the config from HF
model = Messis.from_pretrained('crop-classification/messis', cache_dir='./hf_cache/')

config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

TemporalViTEncoder | Warning: Unexpected keys in the state dict: ['mask_token', 'decoder_pos_embed', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder_blocks.0.norm1.weight', 'decoder_blocks.0.norm1.bias', 'decoder_blocks.0.attn.qkv.weight', 'decoder_blocks.0.attn.qkv.bias', 'decoder_blocks.0.attn.proj.weight', 'decoder_blocks.0.attn.proj.bias', 'decoder_blocks.0.norm2.weight', 'decoder_blocks.0.norm2.bias', 'decoder_blocks.0.mlp.fc1.weight', 'decoder_blocks.0.mlp.fc1.bias', 'decoder_blocks.0.mlp.fc2.weight', 'decoder_blocks.0.mlp.fc2.bias', 'decoder_blocks.1.norm1.weight', 'decoder_blocks.1.norm1.bias', 'decoder_blocks.1.attn.qkv.weight', 'decoder_blocks.1.attn.qkv.bias', 'decoder_blocks.1.attn.proj.weight', 'decoder_blocks.1.attn.proj.bias', 'decoder_blocks.1.norm2.weight', 'decoder_blocks.1.norm2.bias', 'decoder_blocks.1.mlp.fc1.weight', 'decoder_blocks.1.mlp.fc1.bias', 'decoder_blocks.1.mlp.fc2.weight', 'decoder_blocks.1.mlp.fc2.bias', 'decoder_blocks.2.norm1.weight', 'decoder

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [19]:
# get first few weights from model.model.prithvi:
messis_from_pretrained_weights = model.model.prithvi.blocks[0].attn.qkv.weight
messis_from_pretrained_weights = messis_from_pretrained_weights.to(0)
messis_from_pretrained_weights

tensor([[ 0.0125, -0.0556, -0.1178,  ...,  0.0313,  0.0138, -0.0571],
        [ 0.0589,  0.0056,  0.0024,  ...,  0.0440, -0.0671, -0.0293],
        [ 0.0293,  0.0323,  0.1029,  ..., -0.0538,  0.0750, -0.0168],
        ...,
        [-0.0691,  0.0143, -0.0396,  ...,  0.0110,  0.0111,  0.0053],
        [-0.0213, -0.0075,  0.0199,  ...,  0.0025, -0.0709,  0.0018],
        [-0.0829,  0.0343,  0.0246,  ...,  0.0382, -0.0153,  0.0063]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)

### Expectation: these weights directly loaded from Prithvi .pt checkpoint should be the ones present when loading using pretrained="./prithvi/models/Prithvi_100M.pt"

In [20]:
# Load pt file independently ./prithvi/models/Prithvi_100M.pt
weights = torch.load('./prithvi/models/Prithvi_100M.pt')
weights.keys()

original_prithvi_weights = weights['blocks.0.attn.qkv.weight']

# to GPU
original_prithvi_weights = original_prithvi_weights.to(0)
original_prithvi_weights

tensor([[ 0.0119, -0.0555, -0.1045,  ...,  0.0349,  0.0202, -0.0542],
        [ 0.0572,  0.0054, -0.0105,  ...,  0.0428, -0.0756, -0.0308],
        [ 0.0295,  0.0335,  0.1179,  ..., -0.0521,  0.0840, -0.0158],
        ...,
        [-0.0553,  0.0117, -0.0415,  ...,  0.0126,  0.0132,  0.0127],
        [-0.0171, -0.0060,  0.0185,  ...,  0.0019, -0.0689,  0.0041],
        [-0.0943,  0.0341,  0.0188,  ...,  0.0410, -0.0187,  0.0078]],
       device='cuda:0')

### Weights in the safetensors checkpoint

In [21]:
path = "hf_cache/models--crop-classification--messis/snapshots/4f9a588c51c8dbaf2dcff67223d5908e4bf710a2/model.safetensors"

hf_safetensors_weights = None

with safe_open(path, framework="pt", device=0) as f:
    weights_safetensors = tensor_slice = f.get_slice("model.prithvi.blocks.0.attn.qkv.weight")
    print(tensor_slice[:3])

    hf_safetensors_weights = tensor_slice[:]

tensor([[ 0.0125, -0.0556, -0.1178,  ...,  0.0313,  0.0138, -0.0571],
        [ 0.0589,  0.0056,  0.0024,  ...,  0.0440, -0.0671, -0.0293],
        [ 0.0293,  0.0323,  0.1029,  ..., -0.0538,  0.0750, -0.0168]],
       device='cuda:0')


In [11]:
type(hf_safetensors_weights), type(messis_from_pretrained_weights)

(torch.Tensor, torch.Tensor)

### check if the weights were loaded as expected

In [22]:
# for the case that the backbone is was trained, we expect:
# messis_from_pretrained_weights != original_prithvi_weights
# messis_from_pretrained_weights == hf_safetensors_weights
print(f" Is from_pt_weights_inizialized = from_prithvi_weights {torch.allclose(messis_from_pretrained_weights, original_prithvi_weights)}") # False
print(f" Is from_pt_weights_inizialized = from_safetensors_weights {torch.allclose(messis_from_pretrained_weights, hf_safetensors_weights)}") # True

 Is from_pt_weights_inizialized = from_prithvi_weights False
 Is from_pt_weights_inizialized = from_safetensors_weights True
